# Generate paper figures

The following code will recreate the figures in our paper *From a Large Language Model to Three-Dimensional Sentiment*. 

In [ ]:
import os
import sys
import time
import pandas as pd
import numpy as np
import json
from sentiment3d import Sentiment3D
from utils import (
    load_wan_ratings,
    sentiment_from_logits,
    df_corr,
    get_corr,
    get_stats,
    separate_utterances,
    generate_logits,
    map_cols,
)
from plotting import (
    plotly_settings,
    plot_heatmap,
    plot_combined_distributions,
    plot_timeseries,
)

plotly_settings()

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth", 1000)

# valid figure formats are jpg, png, svg, and pdf. svg and pdf will produce 
# the highest quality vector graphics, but for slides png is best.
FIG_FMT = "png"
FIG_SCALE = 2 if FIG_FMT in ("png", "jpg") else 1

FIGDIR = f"./figures"
os.makedirs(FIGDIR, exist_ok=True)
print(f"figures will be saved to {FIGDIR}")

## Load human rating data

In [ ]:
hdf = load_wan_ratings()
hdf.head(2)

## Create subsets of rating data

* wdf is the full set of NRC/Warriner ratings in wide format
* swdf is the smaller set of ratings where NRC, Warriner and ANEW overlap

In [ ]:
wdf = hdf.loc[hdf.source != "anew"].pivot(index="word", columns="source").copy()
wdf.columns = ["_".join(c) for c in wdf.columns]
wdf = wdf.loc[:, [c for c in wdf.columns if "std" not in c]]
wdf.dropna(inplace=True)
wdf = map_cols(wdf)

In [ ]:
swdf = hdf.pivot(index="word", columns="source").copy()
swdf.columns = ["_".join(c) for c in swdf.columns]
swdf = swdf.loc[:, [c for c in swdf.columns if "std" not in c]]
swdf.dropna(inplace=True)
swdf = map_cols(swdf)

## Load sentiment model definition

In [ ]:
# you can modify the model by changing the anchor words
with open("./anchor_spec.json") as fp:
    model = json.load(fp)

model

## Run sentiment model on human ratings data
To compate our sentiment model with human sentiment ratings, we need to score all the words in the human rating data. By default we use a set of precomputed logits for efficiency. But if you want to add anchor points to the model and/or add to the set of words for comparison, you can set RECOMPUTE_LOGITS to True in the following cell. This will recompute all the logits and should take about 1-2 hours on a modern gpu.

In [ ]:
RECOMPUTE_LOGITS = False

if RECOMPUTE_LOGITS:
    utterances = wdf.index.tolist()
    logit_df = generate_logits(utterances=utterances, model=model)
else:
    # Setting logit_df to None will just use the saved logits file in the next cell
    logit_df = None

## Heatmap for all words

In [ ]:
sentdf, anchors = sentiment_from_logits(model, wdf.index, logit_df=logit_df)
sentdf.columns = [c.capitalize() + " VAC" for c in sentdf.columns]
tmpdf = wdf.join(sentdf)
KEEP_COLS = [f"{s} {c}" for s in ["Valence", "Arousal", "Confidence"] for c in ["NRC", "Warr", "VAC"]]
tmpdf = tmpdf[KEEP_COLS]
# IS THIS NEEDED?
#map_cols(tmpdf)
stat_rdf, stat_pdf, n, all_stats = get_stats(tmpdf, ["NRC", "Warr"])

r, p, ndf = df_corr(tmpdf)
n = np.nanmin(ndf)
fig = plot_heatmap(r, font_sz=16, size=(700, 700))
fig.write_image(f"{FIGDIR}/vac_heatmap_wn_{n}.{FIG_FMT}", format=FIG_FMT, scale=FIG_SCALE)
fig

## Heatmap for 1023 WAN words

In [ ]:
sentdf, anchors = sentiment_from_logits(model=model, utterances=swdf.index)
sentdf.columns = [c.capitalize() + " VAC" for c in sentdf.columns]
tmpdf = swdf.join(sentdf)
KEEP_COLS = [f"{s} {c}" 
             for s in ["Valence", "Arousal", "Confidence"] 
             for c in ["NRC", "Warr", "ANEW", "VAC"]]
tmpdf = tmpdf[KEEP_COLS]
r, p, ndf = df_corr(tmpdf)
n = np.nanmin(ndf)

fig = plot_heatmap(r, font_sz=16, size=(800, 850),blocksize=4)
fig.write_image(f"{FIGDIR}/vac_heatmap_wan_{n}.{FIG_FMT}", format=FIG_FMT, scale=FIG_SCALE) 
fig

## Carl Roger and Gloria therapy session

In [ ]:
carl_gloria_df = pd.read_csv("data/carl_and_gloria.csv", sep="\t", index_col=0,)
carl_gloria_df.head()

In [ ]:
utterance_df = separate_utterances(carl_gloria_df)
utterance_df.head()

## Compute sentiment of each utterance
This may take some time if you don't have a GPU. E.g., on a macbook with an M1 Max, it takes about 10 minutes if you use the CPU and about 2-3 minutes if you use the Apple silicon GPU (mps).

In [ ]:
start = time.time()
s3 = Sentiment3D()
utterances = utterance_df["utterance"].to_list()
sentiment_dict = s3(utterances)
print(f"Computed sentiment for {len(utterances)} utterances in {(time.time()-start)/60:0.2f} minutes.")

In [ ]:
utt_res = pd.json_normalize(sentiment_dict, max_level=2)
sent_df = utterance_df.merge(utt_res, left_index=True, right_index=True, validate="1:1")
sent_df.head()

In [ ]:
therapist_df = sent_df[utterance_df['speaker']=='Therapist'].reset_index(drop=True)
patient_df = sent_df[utterance_df['speaker']=='Patient'].reset_index(drop=True)

fig = plot_combined_distributions(sample1=therapist_df, 
                                  sample2=patient_df, 
                                  xlabels=['valence', 'arousal', 'confidence'], 
                                  group_labels=['Therapist', 'Patient'])
fig.write_image(f"{FIGDIR}/carl_roger_combined_distributions.{FIG_FMT}", 
                format=FIG_FMT, scale=FIG_SCALE)
fig.show()

## Timeseries plot

Next we plot the sentiment timeseries for the therapist and patient in the therapy session. For this, we aggregate the sentiment across all the utterances the comprise each talk-turn.

In [ ]:
# group by speaker and aggregate using mean
sent_talk_turn_df = (
    sent_df.groupby(
        (sent_df["speaker"] != sent_df["speaker"].shift()).cumsum()
    )
    .agg(
        {
            "speaker": "first",
            "utterance": " ".join,
            "valence": "mean",
            "arousal": "mean",
            "confidence": "mean",
        }
    )
    .reset_index(drop=True)
)
sent_talk_turn_df.head()

In [ ]:
therapist_df = sent_talk_turn_df[(sent_talk_turn_df["speaker"] == "Therapist")].reset_index(drop=True)
patient_df = sent_talk_turn_df[(sent_talk_turn_df["speaker"] == "Patient")].reset_index(drop=True)

In [ ]:
span_n = 3

samples1 = [
    [therapist_df.index, therapist_df["valence"].ewm(span=span_n).mean()],
    [therapist_df.index, therapist_df["arousal"].ewm(span=span_n).mean()],
    [therapist_df.index, therapist_df["confidence"].ewm(span=span_n).mean()],
]
samples2 = [
    [patient_df.index, patient_df["valence"].ewm(span=span_n).mean()],
    [patient_df.index, patient_df["arousal"].ewm(span=span_n).mean()],
    [patient_df.index, patient_df["confidence"].ewm(span=span_n).mean()],
]

In [ ]:
fig = plot_timeseries(samples1, 
                      samples2, 
                      xlabels=["valence", "arousal", "confidence"], 
                      group_labels=["Therapist", "Patient"])
fig.write_image(f"{FIGDIR}/carl_roger_combined_timeseries.{FIG_FMT}", 
                format=FIG_FMT, scale=FIG_SCALE)
fig